## 資料準備

In [ ]:
# 匯入必要套件
import tensorflow as tf
import numpy as np

# 匯入資料集
from keras.datasets import cifar10
# 資料集切割成訓練與測試資料
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

# 資料大小
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 

In [ ]:
# 對資料做歸一化處理
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

# 對資料樣本作類別標籤(OneHot Code)
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)

# 類別標籤大小 (測試資料筆數:類別總筆數)
y_label_test_OneHot.shape

## 模型方法

In [ ]:
# 匯入搭建神經網路模型套件
from keras import models
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# 定義建立模型方法 (預設參數: 神經網路層數=2, 步數=1, 激活函數='relu', 隨機丟參率=0.25, 優化器='adam')
def create_model(layers=2, filters=32, stride=1, activation='relu', drop_rate=0.25, optimizer='adam'):
  # 建立模型
  model = Sequential()

  for i in range(layers):
    # 添加卷積層和池化層(i+1)
    model.add(Conv2D(filters=filters, 
                    kernel_size=(3,3), 
                    strides=(stride, stride),
                    padding='same', 
                    activation=activation, 
                    input_shape=(32,32,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(drop_rate))

  # 添加平坦層
  model.add(Flatten())
  # 添加全連接層
  model.add(Dense(1024, activation))
  model.add(Dropout(drop_rate))
  # 添加輸出層 (分成10類，激活函數='softmax')
  model.add(Dense(10, activation='softmax'))

  # 編譯模型
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

  return model

## 準確率/損失率

In [ ]:
import matplotlib.pyplot as plt
# # 顯示準確率曲線
# def show_acc_history(history, train_acc,test_acc):
#     plt.figure(figsize=(8, 6))
#     plt.plot(history.history[train_acc])
#     plt.plot(history.history[test_acc])
#     plt.title('Train History')
#     plt.ylabel('Accuracy')
#     plt.xlabel('Epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()

# # 顯示損失率曲線
# def show_loss_history(history, train_loss,test_loss):
#     plt.figure(figsize=(8, 6))
#     plt.plot(history.history[train_loss])
#     plt.plot(history.history[test_loss])
#     plt.title('Train History')
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()

In [ ]:
# 定義繪製訓練準確率與損失率方法
def show_history(history, train_acc, test_acc, train_loss, test_loss):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # 顯示準確率曲線
    ax1.plot(history.history[train_acc])
    ax1.plot(history.history[test_acc])
    ax1.set_title('Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['train', 'test'], loc='upper left')

    # 顯示損失率曲線
    ax2.plot(history.history[train_loss])
    ax2.plot(history.history[test_loss])
    ax2.set_title('Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['train', 'test'], loc='upper left')

    plt.tight_layout()
    plt.show()

## 模型訓練

#### **原始參數**
```
layers=2, filters=32, stride=1, activation='relu', drop_rate=0.25, optimizer='adam'
```
```
batch_size=32
```

In [ ]:
# 建立模型(原始參數)
model = create_model()
# 模型總表
print(model.summary())

# validation_data：使用固定的驗證資料集
# validation_split：使用訓練資料的一部分作為驗證資料集
history = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_data=(x_img_test_normalize, y_label_test_OneHot), epochs=10, batch_size=32, verbose=1)

# 繪製準確率 / 損失率
show_history(history,'accuracy','val_accuracy', 'loss', 'val_loss')

#### **調整濾鏡參數(filters)**
```
layers=2, filters=[32, 64, 128], stride=1, activation='relu', drop_rate=0.25, optimizer='adam'
```

``` filters = 32 ``` 時， 效果最佳

In [ ]:
# # # 建立模型 (filters=32, 64, 128)
# ### filters = 32 效果最佳

# filters = [32, 64, 128]

# for i in range(len(filters)):
#   model = create_model(filters=filters[i])
#   result = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_data=(x_img_test_normalize, y_label_test_OneHot), epochs=10, batch_size=32, verbose=1)
#   show_acc_history(result,'accuracy','val_accuracy')
#   show_loss_history(result, 'loss', 'val_loss')

#### **調整激活函數(activation)**
```
layers=2, filters=32, stride=1, activation=['relu', 'softmax', 'tanh', 'sigmoid', 'LeakyReLU'], drop_rate=0.25, optimizer='adam'
```

``` activation = 'LeakyReLU' ``` 時，效果最佳

In [ ]:
# # 更改激活函數 (activation='relu', 'softmax', 'tanh', 'sigmoid', 'LeakyReLU')
# ### 'relu' 與 'LeakyReLU' 表現較佳  -> 'LeakyReLU' 稍好一些

# activations = [tf.keras.activations.relu, 
#               #  tf.keras.activations.softmax, 
#               #  tf.keras.activations.tanh, 
#               #  tf.keras.activations.sigmoid, 
#                tf.keras.layers.LeakyReLU(alpha=0.01)]

# for i in range(len(activations)):
#   model = create_model(activation=activations[i])
#   result = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_data=(x_img_test_normalize, y_label_test_OneHot), epochs=10, batch_size=32, verbose=1)
#   show_acc_history(result,'accuracy','val_accuracy')
#   show_loss_history(result, 'loss', 'val_loss')

#### **調整優化器(optimizer)**
```
layers=2, filters=32, stride=1, activation='relu', drop_rate=0.25, optimizer=['adam', 'SGD', 'RMSProp', 'Adagrad', 'Adadelta']
```

``` optimizer = 'adam' ``` 時，效果最佳

In [ ]:
# # 更改優化器 (optimizer='adam', 'SGD', 'RMSProp', 'Adagrad', 'Adadelta')
# ### 'adam', 'RMSProp' 表現較佳  -> 'adam' 表現稍好

# optimizers = [tf.keras.optimizers.Adam()]
#               # tf.keras.optimizers.SGD(),
#               # tf.keras.optimizers.RMSprop(),
#               # tf.keras.optimizers.Adagrad(),
#               # tf.keras.optimizers.Adadelta()

# for i in range(len(optimizers)):
#   model = create_model(optimizer=optimizers[i])
#   result = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_data=(x_img_test_normalize, y_label_test_OneHot), epochs=10, batch_size=32, verbose=1)
#   show_acc_history(result,'accuracy','val_accuracy')
#   show_loss_history(result, 'loss', 'val_loss')

#### **調整步幅(stride)**
```
layers=2, filters=32, stride=[1,2], activation='relu', drop_rate=0.25, optimizer='adam'
```

``` stride = 1 ``` 時，效果最佳

In [ ]:
# # 更改步幅 (stride=1, 2)
# ### stride = 1 效果較佳

# # strides = [1, 2]


# for i in range(len(strides)):
#   model = create_model(stride=strides[i])
#   result = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_data=(x_img_test_normalize, y_label_test_OneHot), epochs=10, batch_size=32, verbose=1)
#   show_acc_history(result,'accuracy','val_accuracy')
#   show_loss_history(result, 'loss', 'val_loss')

#### **調整卷積層數(layers)**
```
layers=range(2, 5), filters=32, stride=[1,2], activation='relu', drop_rate=0.25, optimizer='adam'
```

``` layers = 2 ``` 時，效果最佳

In [ ]:
# 更改卷積層數 (layers=range(2, 5))
### layers=2 時效果最佳

# for i in range(2, 5):
#   model = create_model(layers=i)
#   result = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_data=(x_img_test_normalize, y_label_test_OneHot), epochs=10, batch_size=32, verbose=1)
#   show_acc_history(result,'accuracy','val_accuracy')
#   show_loss_history(result, 'loss', 'val_loss')

#### **調整Dropout參數**
```
layers=2, filters=32, stride=[1,2], activation='relu', drop_rate=range(0.2, 0.5, 0.05), optimizer='adam'
```

``` drop_rate = 0.3 ``` 時，效果最佳

In [ ]:
# 更改Dropout (drop_rate=range(0.2, 0.5, 0.05))
### drop_rate = 0.3 > 0.25 > 0.2 效果佳對比

drop_rate = np.arange(0.2, 0.5, 0.05)

# for rate in drop_rate:
#   model = create_model(drop_rate=rate)
#   result = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_data=(x_img_test_normalize, y_label_test_OneHot), epochs=10, batch_size=32, verbose=1)
#   show_acc_history(result,'accuracy','val_accuracy')
#   show_loss_history(result, 'loss', 'val_loss')

#### **調整批量(batch_size)**
```
layers=2, filters=32, stride=[1,2], activation='relu', drop_rate=0.25, optimizer='adam'
```
```
batch_size = [16, 32, 64]
```

``` batch_size = 64``` 時，效果最佳

In [ ]:
# 更改批量 (batch_size=[16, 32, 64])
### batch_size = 64 > 32 > 16

batch_size = [16, 32, 64]

# for batch in batch_size:
#   model = create_model()
#   result = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_data=(x_img_test_normalize, y_label_test_OneHot), epochs=10, batch_size=batch, verbose=1)
#   show_acc_history(result,'accuracy','val_accuracy')
#   show_loss_history(result, 'loss', 'val_loss')

#### **影像擴增**

執行結果沒有比較好 --> 不予採用

In [ ]:
# # 影像擴增

# # 定義資料擴增方法
# def data_augmentation(image, label):
#     # 隨機水平翻轉
#     image = tf.image.random_flip_left_right(image)
    
#     # 隨機旋轉
#     image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

    
#     # 隨機調整亮度
#     image = tf.image.random_brightness(image, max_delta=0.2)
    
#     # 隨機調整對比度
#     image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
    
#     return image, label

# # 建立訓練資料集
# train_dataset = tf.data.Dataset.from_tensor_slices((x_img_train_normalize, y_label_train_OneHot))
# train_dataset = train_dataset.shuffle(50000).map(data_augmentation).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# # 建立測試資料集
# test_dataset = tf.data.Dataset.from_tensor_slices((x_img_test_normalize, y_label_test_OneHot))
# test_dataset = test_dataset.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# model = create_model(drop_rate=0.3, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
# result = model.fit(train_dataset, epochs=10, validation_data=test_dataset, batch_size=32, verbose=1)

# show_acc_history(result,'accuracy','val_accuracy')
# show_loss_history(result, 'loss', 'val_loss')

In [ ]:
# # 評估模型
# scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
# print('Test loss:', scores[0])
# print('Test accuracy:', scores[1])

#### Filter調參：使用網格搜尋(Grid Search) --> 非常不現實的決定

In [ ]:
# KerasClassifier 可以將模型當成scikit-learn中的估計器(estimator)來使用，以便進行機器學習中的交叉驗證和超參數調整

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# # 將模型包裝成 KerasClassifier
# model = KerasClassifier(build_fn=create_model, verbose=0)
# # 定義要調整的 filter 參數範圍
# param_grid = {'filters_1': [32, 64, 128], 'filters_2': [32, 64, 128]}
# # 使用 GridSearchCV 進行自動調參
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
# grid_result = grid.fit(x_img_train_normalize, y_label_train_OneHot)

# # 輸出最佳參數組合和對應的準確度
# print("Best Parameters: ", grid_result.best_params_)
# print("Best Accuracy: ", grid_result.best_score_)

#### 貝葉斯優化：卷積層數(Layers)與濾鏡數量(Filters) --> 運算資源不足 = 不現實的決定

In [ ]:
!pip install optuna

In [ ]:
# import optuna
# from sklearn.metrics import accuracy_score

# def optimize(trial):
#   # 定義超參數搜索範圍
#   n_layers = trial.suggest_int('n_layers', 2, 5)

#   model = Sequential()
#   # 添加卷積層
#   for _ in range(n_layers):
#       n_filters = trial.suggest_int('n_filters', 32, 128)
#       model.add(Conv2D(n_filters, kernel_size=(3, 3), activation='relu', padding='same'))
#       model.add(MaxPooling2D(pool_size=(2, 2)))
#       model.add(Dropout(rate=0.25))

#   # 添加平坦層
#   model.add(Flatten())
#   # 添加全連接層
#   model.add(Dense(1024, activation='relu'))
#   model.add(Dropout(rate=0.25))
#   # 添加輸出層
#   model.add(Dense(10, activation='softmax'))

#   # 編譯模型
#   model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  
#   # 訓練模型
#   model.fit(x_img_train_normalize, y_label_train_OneHot, validation_data=(x_img_test_normalize, y_label_test_OneHot), epochs=10, batch_size=32, verbose=1)  
#   # 計算驗證集上的準確度
#   y_pred = model.predict(x_img_test_normalize)
#   y_pred = np.argmax(y_pred, axis=1)
#   accuracy = accuracy_score(np.argmax(y_label_test_OneHot, axis=1), y_pred)
  
#   return accuracy

In [ ]:
# # 創建Optuna優化器並執行優化
# study = optuna.create_study(direction='maximize')
# study.optimize(optimize, n_trials=100)

# # 取得最佳的超參數組合
# best_params = study.best_params
# best_accuracy = study.best_value

# print('Best Parameters:', best_params)
# print('Best Accuracy:', best_accuracy)

In [ ]:
# # 設定參數範圍
# param_grid = {'stride': [1, 2]}

# # 建立Keras分類器
# model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# # 使用GridSearchCV進行自動化調參
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
# grid_result = grid.fit(x_img_train_normalize, y_label_train_OneHot)

# # 印出最佳結果
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
# import optuna
# import tensorflow as tf
# from tensorflow import keras
# from keras.datasets import cifar10
# from keras.utils import np_utils
# from keras.preprocessing.image import ImageDataGenerator
# from sklearn.model_selection import train_test_split

# # 載入CIFAR-10資料集
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()

# # 對資料做歸一化處理
# x_train = x_train.astype('float32') / 255.0
# x_test = x_test.astype('float32') / 255.0

# # 對資料樣本作類別標籤(OneHot Code)
# num_classes = 10
# y_train = np_utils.to_categorical(y_train, num_classes)
# y_test = np_utils.to_categorical(y_test, num_classes)

# # 分割訓練集和驗證集
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# def create_model(trial):
#     model = keras.Sequential()
#     model.add(keras.layers.Conv2D(trial.suggest_int('filters_1', 16, 64), kernel_size=(3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
#     model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(trial.suggest_int('stride_1', 1, 2))))
    
#     for i in range(trial.suggest_int('num_layers', 1, 3)):
#         model.add(keras.layers.Conv2D(trial.suggest_int(f'filters_{i+2}', 16, 64), kernel_size=(3, 3), activation='relu', padding='same'))
#         model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(trial.suggest_int(f'stride_{i+2}', 1, 2))))

#     model.add(keras.layers.Flatten())
#     model.add(keras.layers.Dense(trial.suggest_int('units', 64, 256), activation='relu'))
#     model.add(keras.layers.Dropout(trial.suggest_float('dropout', 0.1, 0.5)))
#     model.add(keras.layers.Dense(num_classes, activation='softmax'))
    
#     return model

# def objective(trial):
#     model = create_model(trial)
    
#     # 定義優化器
#     optimizer = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])
    
#     # 定義影像擴增
#     data_augmentation = trial.suggest_categorical('data_augmentation', [False, True])
    
#     # 編譯模型
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
#     if data_augmentation:
#         # 影像擴增
#         datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
#         datagen.fit(x_train)
#         history = model.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=10, validation_data=(x_val, y_val), verbose=0)
#     else:
#         history = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val), verbose=0)
    
#     # 計算驗證集的準確度
#     val_accuracy = history.history['val_accuracy'][-1]
    
#     return val_accuracy

# # 創建Optuna Study物件
# study = optuna.create_study(direction='maximize')

# # 最大試驗次數
# n_trials = 50

# # 開始進行參數調整
# study.optimize(objective, n_trials=n_trials)

# # 印出最佳參數組合
# print('Best Parameters: ')
# best_params = study.best_params
# for key, value in best_params.items():
#     print(f'{key}: {value}')

## 經過調參

In [ ]:
# 更換成調參結果，建立模型
model = create_model(drop_rate=0.3, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
# 模型總表
print(model.summary())

# validation_data：使用固定的驗證資料集
# validation_split：使用訓練資料的一部分作為驗證資料集
history = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_data=(x_img_test_normalize, y_label_test_OneHot), epochs=10, batch_size=64, verbose=1)

# 顯示準確率 / 損失率
show_history(history,'accuracy','val_accuracy', 'loss', 'val_loss')

## Graph (繪製神經網路圖)

In [ ]:
import tensorflow as tf
import pydotplus
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
!pip install ann_visualizer

In [ ]:
from ann_visualizer.visualize import ann_viz
# 可能要等幾秒才會跑出來(顯示在colab files中)
ann_viz(model, filename='network.gv', title='Network')